# Gesture Annotation Notebook |

##### <strong>Author:</strong> <u>Walter Dych</u> <em>(walterpdych@gmail.com)</em>
##### <strong>Edits/Documentation:</strong> <u>Karee Garvin</u> <em>(kgarving@fas.harvard.edu)</em>

In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [ ]:
data = pd.read_csv('C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/SPEED_FILES/5012_I_processed_data.csv')
data.head()

In [ ]:
MAX_S_LENGTH = 60
MAX_STEADY_LENGTH = 50
THRESHOLD = 0.4
COLORS = {'Approach': 'rgba(231,107,243,0.2)', 'Steady': 'rgba(107,174,214,0.2)', 'R': 'rgba(127,127,127,0.2)', '': 'rgba(255,255,255,0.2)'}

In [ ]:
def generate_annotations(data): # Generate annotations based on speed_smooth data.
    speed_smooth = list(map(float, data['speed_smooth']))
    state = [''] * len(speed_smooth)
    last_stroke_type = None
    start_index = None
    steady_start = None
    steady_end = None
    apexes = [''] * len(speed_smooth)

    # Iterate over the speed_smooth array to generate the annotations
    for i in range(1, len(speed_smooth)):
        if speed_smooth[i] > THRESHOLD and speed_smooth[i-1] <= THRESHOLD:
            if last_stroke_type is None or last_stroke_type == 'Steady' or last_stroke_type == 'R':
                last_stroke_type = 'Approach'
                start_index = i
            elif last_stroke_type == 'Approach' and i - start_index >= MAX_S_LENGTH:
                last_stroke_type = None
        elif speed_smooth[i] <= THRESHOLD and speed_smooth[i-1] > THRESHOLD:
            if last_stroke_type == 'Approach':
                last_stroke_type = 'Steady'
                start_index = i
                steady_start = i
            elif last_stroke_type == 'R':
                last_stroke_type = None
                start_index = None
        elif last_stroke_type == 'Steady' and speed_smooth[i] > THRESHOLD and speed_smooth[i-1] <= THRESHOLD:
            if steady_end is not None:
                last_stroke_type = 'R'
                start_index = i

        if last_stroke_type == 'Steady' and steady_start is None:
            steady_start = i
        elif steady_start is not None and last_stroke_type != 'Steady':
            steady_end = i
            min_speed_index = np.argmin(speed_smooth[steady_start:i]) + steady_start
            apexes[min_speed_index] = 'AX'
            steady_start = None
            if speed_smooth[i] > THRESHOLD and speed_smooth[i-1] <= THRESHOLD:
                last_stroke_type = 'R'
                start_index = i
        elif last_stroke_type == 'Steady' and i - start_index >= MAX_STEADY_LENGTH:
            last_stroke_type = None
            start_index = None
            steady_start = None
        state[i] = last_stroke_type if last_stroke_type is not None else ''

    data = pd.DataFrame({
        'time_ms': data['time_ms'],
        'speed_unsmooth': data['speed_unsmooth'],
        'speed_smooth': data['speed_smooth'],
        'state': state, 
        'apex': apexes
    })

    data['Gesture Phase'] = data['state'].apply(lambda x: 'S' if x in ['Steady', 'Approach'] else ('R' if x == 'R' else ''))
    return data

data.head()

In [ ]:
data = generate_annotations(data)
data.head()

In [ ]:
data.to_csv('C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/ANNOTATIONS/5012_I_annotations.csv', index=False)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=data['time_ms'], 
    y=data['speed_smooth'], 
    mode='lines',
    name='Speed curve'
))

data['State_Change'] = data['state'].shift() != data['state']
data['Group'] = data['State_Change'].cumsum()
interval_df = data.groupby(['Group', 'state']).agg(Start=('time_ms', 'min'), End=('time_ms', 'max')).reset_index()

shapes = []
for _, row in interval_df.iterrows():
    shapes.append(dict(
        type="rect",
        xref="x", yref="paper",
        x0=row['Start'], x1=row['End'],
        y0=0, y1=1,
        fillcolor=COLORS[row['state']],
        opacity=0.85,
        layer="below",
        line_width=0,
    ))

for state in COLORS.keys():
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=15, color=COLORS[state]),
        showlegend=True,
        name=state,
    ))

apex_df = data[data['apex'] == 'AX']
fig.add_trace(go.Scatter(
    x=apex_df['time_ms'], 
    y=apex_df['speed_smooth'], 
    mode='markers',
    name='apexes'
))

fig.update_xaxes(title_text='time')
fig.update_yaxes(title_text='Speed (px/ms)')
fig.update_layout(showlegend=True, shapes=shapes)

fig.show()